<a href="https://colab.research.google.com/github/ldj7672/LLM-Tutorials/blob/main/examples/OpenAI_RAG_Llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OpenAI와 Llama-index를 이용한 RAG 실습**

**OpenAI**와 **Llama-index**를 이용한, 조금 더 다양한 기능의 **RAG** 기능을 구현해보는 실습 코드입니다.

## **1. 환경 세팅**
- 필요 라이브러리 설치
- API Key 입력
- 구글 드라이브 마운트

In [ ]:
!pip install -q openai llama-index llama-index-vector-stores-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.1 MB/s eta 0:00:0

In [ ]:
from getpass import getpass
import os
from openai import OpenAI
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding

In [ ]:
# Colab에 API 키를 안전하게 입력받기
api_key = getpass("OpenAI API 키를 입력하세요: ")
os.environ['OPENAI_API_KEY'] = api_key

OpenAI API 키를 입력하세요: ··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **2. 기본 예제**
- 지정된 디렉토리에서 문서를 읽고, 이 문서들로부터 `VectorStoreIndex`를 생성하고, 이 인덱스를 쿼리 엔진으로 변환하여 입력된 쿼리와 관련된 정보를 검색하는 **가장 기본적인 예제**
- 별도의 임베딩 모델이나 ChromaDB 같은 외부 벡터 저장소를 지정하지 않기 때문에 세밀한 조정은 불가능.

In [ ]:
# RAG 참조 문서 디렉토리 지정 (각자 지정 필요)
documents_dir = '/content/drive/MyDrive/코드 예제/LLM/ref_docs_2'

In [ ]:
# 문서를 읽어와 인덱스를 생성하고, 간단한 쿼리 엔진으로 검색하는 기본 예제입니다.
documents = SimpleDirectoryReader(documents_dir).load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# 인덱스를 사용해 간단한 쿼리 엔진을 생성합니다.
query_engine = index.as_query_engine()

response = query_engine.query("KBO 한국 시리즈에서 우승하지 못한 팀을 알려줘")
display(Markdown(f"{response}"))

삼성 라이온즈


## **3. ChromaDB와 OpenAI Embedding 모델 사용 예제**

- 벡터 저장소로 ChromaDB를 통합하고 `OpenAIEmbedding`을 사용하여 커스텀 임베딩 모델을 정의
- 메모리 상에 ChromaDB 클라이언트와 컬렉션을 초기화하고, `ChromaVectorStore`를 설정하여 데이터를 저장한 후 해당 저장소와 임베딩 모델을 사용하여 인덱스를 생성
- 기본 예제와 달리, **임베딩 모델**과 **벡터 저장소**를 명시적으로 설정하여 **검색 정확도를 높일 수 있는 방법**

In [ ]:
# ChromaDB 클라이언트를 메모리상에 생성하고 새로운 컬렉션을 만듭니다.
chroma_client = chromadb.EphemeralClient()  # 메모리 상에 저장 (세션 간 데이터 유지가 필요 없다면 이 옵션 사용)
chroma_collection = chroma_client.create_collection("quickstart")  # 컬렉션 생성은 최초 1회만 실행합니다.

In [ ]:
# OpenAI 임베딩 모델을 정의합니다. (필요에 따라 모델을 변경할 수 있습니다.)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# 문서들을 지정된 디렉토리에서 불러옵니다.
documents = SimpleDirectoryReader(documents_dir).load_data()

# ChromaVectorStore를 설정하고 데이터를 저장합니다.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 문서들로부터 인덱스를 생성합니다. 이때 ChromaDB와 임베딩 모델을 사용합니다.
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [ ]:
# 쿼리 엔진을 사용하여 질문에 답변합니다.
query_engine = index.as_query_engine()

# 예시 쿼리: 한국 시리즈에서 우승하지 못한 팀을 알려줘
response = query_engine.query("한국 시리즈에서 우승하지 못한 팀을 알려줘")
display(Markdown(f"{response}"))

키움 히어로즈

## **4. 고급 검색 및 후처리 사용 예제**
- 검색 프로세스에 대해 더 세밀한 제어를 제공하는 방법
- `VectorIndexRetriever`를 설정하여 **`similarity_top_k`** 파라미터 조절
- `get_response_synthesizer()`를 사용하여 검색된 문서와 쿼리를 합치는 합성기를 정의
- `SimilarityPostprocessor`의 **`similarity_cutoff`** 로 결과 필터링

- **`similarity_top_k`**
  - 유사한 문서의 개수를 조절
  - 값이 높으면 더 많은 문서를 검색하지만, 불필요한 정보가 포함될 수 있음
- **`similarity_cutoff`**
  - 검색된 문서 중 유사도가 일정 수준 이상인 것만 포함
  - 값을 낮추면 더 많은 결과를 포함하지만, 정확도가 떨어질 수 있음

In [ ]:
# 리트리버를 설정합니다. 'similarity_top_k'로 유사한 문서의 개수를 조절할 수 있습니다.
retriever = VectorIndexRetriever(
    index=index,  # 기존에 생성한 인덱스를 불러옵니다.
    similarity_top_k=10,  # 유사한 문서 최대 10개를 선택합니다.
)

# 검색된 문서와 쿼리를 합성해주는 합성기를 가져옵니다.
response_synthesizer = get_response_synthesizer()

# 쿼리 엔진을 생성합니다. 후처리 단계에서 'similarity_cutoff'를 설정하여 유사도가 낮은 결과를 필터링할 수 있습니다.
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.1)],  # 유사도가 0.1 이하인 결과는 제외
)

In [ ]:
response = query_engine.query("KBO에서 가장 유명한 사건 3개를 알려줘")
display(Markdown(f"{response}"))

삼청태, 삼성의 대구 유니폼, 그리고 2023 월드 베이스볼 클래식에서의 이강철호의 3대회 연속 1라운드 탈락이 KBO에서 가장 유명한 사건 3개입니다.